WaPo article here:

https://www.washingtonpost.com/local/education/college-major-salary/2020/12/24/cad0f5de-44b3-11eb-b0e4-0f182923a025_story.html

References publication:
    
http://cew.georgetown.edu/wp-content/uploads/CEW-Buyer-Beware.pdf
    
I'm trying to recreate the results from the data. The study appears to have used the 1516-1617 data. I'm basing this on spot checking various data points in the article - these match the 1516-1617 csv file. This notebook will work for most recent cohorts as well, though the numbers change (in many cases 1yr pay is considerably higher in the more recent data set). 

Trying to reproduce these results:

 27 percent of workers with an associate’s degree earn more than the median for workers with a bachelor’s degree

 35 percent of workers with a bachelor’s degree earn more than the median for workers with a master’s degree

 31 percent of workers with a master’s degree earn more than the median for workers with a doctoral degree

 22 percent of workers with a master’s degree earn more than the median for workers with a professional degree.6

I was unable to verify these figures. Code is below. 

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Using the data referenced in the article.

https://collegescorecard.ed.gov/data/
    
Links to:

https://data.ed.gov/dataset/college-scorecard-all-data-files-through-6-2020/resources

Downloaded "All College Scorecard Data Files"

Using Most-Recent-Cohorts-Field-of-Study (downloaded 1/5/2021)

In [2]:
df = pd.read_csv('data/FieldOfStudyData1516_1617_PP.csv')

In [3]:
len(df)

218901

Removing lines without data (PrivacySuppressed). I think this results from cohorts that are small enough that the results are personally identifiable. 

In [4]:
df_1yr = pysqldf("""
SELECT 
    INSTNM, 
    CIPCODE,
    CIPDESC, 
    CREDLEV, 
    CREDDESC, 
    EARN_COUNT_WNE_HI_1YR * 1 AS EARN_COUNT_WNE_HI_1YR, 
    EARN_MDN_HI_1YR * 1.0 AS EARN_MDN_HI_1YR
FROM 
    df
WHERE
    EARN_MDN_HI_1YR <> 'PrivacySuppressed'
ORDER BY EARN_MDN_HI_1YR
""")

In [5]:
pysqldf("SELECT DISTINCT CREDLEV, CREDDESC FROM df_1yr ORDER BY CREDLEV")

,CREDLEV,CREDDESC
0,1,Undergraduate Certificate or Diploma
1,2,Associate's Degree
2,3,Bachelors Degree
3,4,Post-baccalaureate Certificate
4,5,Master's Degree
5,6,Doctoral Degree
6,7,First Professional Degree
7,8,Graduate/Professional Certificate


First, validate a single easy datapoint.

Graduates who earn an associate’s degree from
Pierpont Community and Technical College in West Virginia to become an electrical and
power transmission installer can expect to make \\$6,700 per month (\\$80,400 per year) in
their first year after graduation.

Ever so slightly off, I get \\$80,100

In [6]:
pysqldf("""
SELECT 
    * 
FROM 
    df_1yr 
WHERE 
    INSTNM = 'Pierpont Community and Technical College'
AND 
    CIPDESC = 'Electrical and Power Transmission Installers.'
""")

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_COUNT_WNE_HI_1YR,EARN_MDN_HI_1YR
0,Pierpont Community and Technical College,4603,Electrical and Power Transmission Installers.,2,Associate's Degree,0,80100.0


try one more.

For instance, the  rst-year earnings of a person who is awarded
an associate’s degree in nursing from City University of New York (CUNY) LaGuardia Community College are \\$5,017 per month, which is about \\$800 higher than the median monthly earnings of graduates from master’s degree programs at all institutions.

Confirmed below. 

In [7]:
pysqldf("""
SELECT 
    * 
FROM 
    df_1yr 
WHERE 
    INSTNM = 'CUNY LaGuardia Community College'
AND
    CIPDESC = 'Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.'
AND
    CREDLEV = 2
""")

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_COUNT_WNE_HI_1YR,EARN_MDN_HI_1YR
0,CUNY LaGuardia Community College,5138,"Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.",2,Associate's Degree,24,60200.0


In [8]:
60200.0 / 12

5016.666666666667

Now moving on to the calcs I'mn trying to verify (percent at one degree level at or above the median for a different degree level).

Separate dataframe for each degree level I'll look at. 

In [9]:
df_asoc = pysqldf("SELECT * FROM df_1yr WHERE CREDLEV = 2")
df_bach = pysqldf("SELECT * FROM df_1yr WHERE CREDLEV = 3")
df_ms = pysqldf("SELECT * FROM df_1yr WHERE CREDLEV = 5")
df_prof = pysqldf("SELECT * FROM df_1yr WHERE CREDLEV = 7")
df_doc = pysqldf("SELECT * FROM df_1yr WHERE CREDLEV = 6")

To get median for all graduates, I need to consider the number of graduates in each cohort (I can't take the median for all programs as the median for all students, because there are varying numbers in each cohort).

To do this, I'll take the cumulative sum of the number of graduates in each program ordered by earnings (2nd year out).

In [10]:
df_asoc['num_below'] = df_asoc['EARN_COUNT_WNE_HI_1YR'].cumsum()
df_bach['num_below'] = df_bach['EARN_COUNT_WNE_HI_1YR'].cumsum()
df_ms['num_below'] = df_ms['EARN_COUNT_WNE_HI_1YR'].cumsum()
df_prof['num_below'] = df_prof['EARN_COUNT_WNE_HI_1YR'].cumsum()
df_doc['num_below'] = df_doc['EARN_COUNT_WNE_HI_1YR'].cumsum()

In [11]:
# quick sanity check, visual inspectin to make sure ordering is correct
df_bach

,INSTNM,CIPCODE,CIPDESC,CREDLEV,CREDDESC,EARN_COUNT_WNE_HI_1YR,EARN_MDN_HI_1YR,num_below
0,University of Puerto Rico-Ponce,5109,"Allied Health Diagnostic, Intervention, and Treatment Professions.",3,Bachelors Degree,24,4400.0,24
1,Universidad del Sagrado Corazon,904,Journalism.,3,Bachelors Degree,28,5200.0,52
2,Caribbean University-Bayamon,5108,Allied Health and Medical Assisting Services.,3,Bachelors Degree,50,5500.0,102
3,Caribbean University-Ponce,5108,Allied Health and Medical Assisting Services.,3,Bachelors Degree,50,5500.0,152
4,Universidad Ana G. Mendez-Gurabo Campus,5102,Communication Disorders Sciences and Services.,3,Bachelors Degree,96,6300.0,248
5,University of Puerto Rico-Cayey,5203,Accounting and Related Services.,3,Bachelors Degree,24,6600.0,272
6,Universidad del Sagrado Corazon,907,"Radio, Television, and Digital Communication.",3,Bachelors Degree,43,6800.0,315
7,Caribbean University-Bayamon,4407,Social Work.,3,Bachelors Degree,26,7100.0,341
8,Caribbean University-Carolina,4407,Social Work.,3,Bachelors Degree,26,7100.0,367
9,Caribbean University-Ponce,4407,Social Work.,3,Bachelors Degree,26,7100.0,393


Data is prepped.

I'll try to validate the report figures on the percentage of one degree level earning at or above the median for a different degree level. Starting out with what percentage of Bachelors degree graduates earn more than the median for Masters degree holders. 

First, I'll calculate the median salary for all MS degree recipients.

To do this, I find the median salary for the cohort. As mentione above, this isn't as easy as finding the salary for the median program, since they have differing cohort sizes. I need to find the salary for the median graduate. To do this, I find the salary for the program and cohort closest to the median masters student (using the cumulative count of all students, called "num_below"). 

In [12]:
pysqldf("""
SELECT 
        EARN_MDN_HI_1YR 
    FROM 
        (SELECT 
            *, 
            ABS(num_below - (SELECT MAX(num_below)/2 FROM df_ms)) AS D 
        FROM 
            df_ms
        ORDER BY D 
        LIMIT 1)
""")

,EARN_MDN_HI_1YR
0,54400.0


Now that I have hat number, I can count the number of graduates at a different degree level have a salary above that number, and then divide it by the total number of grads at that degree level to get the percentage.

Problem here... I'm getting ~17% of bachelors grads getting higher salaries than the median masters, which doesn't match the numbers reported in the article. Gotta check this. 

In [13]:
pysqldf("""
SELECT 
    SUM(EARN_COUNT_WNE_HI_1YR) * 1.0 / (SELECT MAX(num_below) FROM df_bach) * 1.0 AS PCT_ABOVE
FROM 
    df_bach 
WHERE 
    EARN_MDN_HI_1YR >= 54400.0
""")

,PCT_ABOVE
0,0.166612


Made it a little more generalizable, to compare any two degree levels this way

A little more explanation of this query.

I want to find the percent of graduates at one degree level who have a 1st year salary at or above the median for a different degree level. 

The data is not reported for each graduate. Instead, the data includes a row for each cohort for each degree program/instution. This row includes the number of graduates and the median 1yr pay for this group. 

To find the medians, I ordered the data by 1yr pay in ascending order, then created a new column that holds the cumulative number of graduates at or below each row (called "num_below", as it provides the number of graduates at or below the median salary for a particular cohort within each degree level). I use this column to find the 1yr salary of the median graduate for each degree level (rather than the median program). 

Now that I have the median 1yr salary for a degree level, I can find the number of graduates at or above this median for graduates from a different degree level (using the same num_row column column). 


The queries in the cell below carry out these calculations - first the median for a degree level ("med"), then the percentage at or above it from a different degree level "above"). 

In [34]:
def pct_at_or_above_median(med, above):

    return pysqldf("""
    WITH med AS 
    (
    SELECT 
        EARN_MDN_HI_1YR 
    FROM 
        (SELECT 
            *, 
            ABS(num_below - (SELECT MAX(num_below)/2 FROM {0})) AS D 
        FROM 
            {0} 
        ORDER BY D 
        LIMIT 1)
    )

    SELECT 
        SUM(EARN_COUNT_WNE_HI_1YR) * 1.0 / (SELECT MAX(num_below) FROM {1}) * 1.0 AS PCT_ABOVE
    FROM 
        {1} 
    WHERE 
        EARN_MDN_HI_1YR >= (SELECT EARN_MDN_HI_1YR FROM med)

    """.format(med, above))

For quick reference, here's a repeat of the results I'm trying to duplicated

- 27 percent of workers with an associate’s degree earn more than the median for workers with a bachelor’s degree
- 35 percent of workers with a bachelor’s degree earn more than the median for workers with a master’s degree
- 31 percent of workers with a master’s degree earn more than the median for workers with a doctoral degree
- 22 percent of workers with a master’s degree earn more than the median for workers with a professional degree.

In [35]:
pct_at_or_above_median('df_bach', 'df_asoc')

,PCT_ABOVE
0,0.177478


In [36]:
pct_at_or_above_median('df_ms', 'df_bach')

,PCT_ABOVE
0,0.166612


In [33]:
pct_at_or_above_median('df_doc', 'df_ms')

,PCT_ABOVE
0,0.154378


In [18]:
pct_at_or_above_median('df_prof', 'df_ms')

,PCT_ABOVE
0,0.283655


### calculatiions based on median program, rather than median graduate

It occurred to me that the calculations in the article might have simply used the median for each program rather than each graduate. I tried these calculations below (I don't think this is the right way to do it, since the cohort size varies, but I thought it might explain the discrepancy). 

These numbers don't match, either, so looks like this doesn't explain why I'm getting different numbers. And like I said above, I don't think this is the right approach to the calcs anyway. 

In [19]:
m_asoc = df_asoc.describe()['EARN_MDN_HI_1YR']['50%']
m_bach = df_bach.describe()['EARN_MDN_HI_1YR']['50%']
m_ms = df_ms.describe()['EARN_MDN_HI_1YR']['50%']
m_doc = df_doc.describe()['EARN_MDN_HI_1YR']['50%']
m_prof = df_prof.describe()['EARN_MDN_HI_1YR']['50%']

In [20]:
print(m_asoc)
print(m_bach)
print(m_ms)
print(m_doc)
print(m_prof)

29400.0
34500.0
50600.0
69150.0
61700.0


In [21]:
def pct_at_or_above_median_program(median, degree):
    return pysqldf("""
    SELECT (COUNT(*) * 1.0) / ((SELECT COUNT(*) FROM {0})*1.0) as pct_above
    FROM {0} WHERE EARN_MDN_HI_1YR >= {1}""".format(degree, median))

In [22]:
pct_at_or_above_median_program(m_bach, 'df_asoc')

,pct_above
0,0.35324


In [23]:
pct_at_or_above_median_program(m_ms, 'df_bach')

,pct_above
0,0.188873


In [24]:
pct_at_or_above_median_program(m_doc, 'df_ms')

,pct_above
0,0.195199


In [25]:
pct_at_or_above_median_program(m_prof, 'df_ms')

,pct_above
0,0.279418
